In [1]:
# Install (if not already in your venv)
!pip install datasets transformers

# Imports
from datasets import load_dataset
from transformers import AutoTokenizer
from pprint import pprint
import random

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


c:\Users\Zayed\Documents\GitHub\FactuAI\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset

# Load the full Multi-News dataset
dataset = load_dataset("alexfabbri/multi_news", trust_remote_code=True)

# Check number of samples
print(f"Train: {len(dataset['train'])}, Validation: {len(dataset['validation'])}, Test: {len(dataset['test'])}")


Generating test split: 100%|██████████| 5622/5622 [00:00<00:00, 6654.76 examples/s]


Train: 44972, Validation: 5622, Test: 5622


In [4]:
# View a random sample from the training set
sample = random.choice(dataset["train"])
print("Document (with ||||| separators):\n")
print(sample["document"][:1000])  # limit to 1000 chars
print("\nSummary:\n")
print(sample["summary"])


Document (with ||||| separators):

Women tolerate an unattractive man up to a point, but beware if he misbehaves. Then they'll easily shun him. So say Jeremy Gibson and Jonathan Gore of the Eastern Kentucky University in the US, after finding that a woman's view of a man is influenced by how handsome and law-abiding he is. Their study in Springer's journal Gender Issues has significance for those using dating sites or doing jury duty. 
 
 Discovering how someone can make a positive first impression is an important field of study because of its role in forming relationships. It is often based on physical appearance and whether someone sticks to social norms or not. Such impressions are made in a flash, but are not always correct. In what is called the "halo effect," people warm up to others with positive characteristics, such as handsomeness. The "devil effect" or "negative halo effect" comes into play when people assume that others possess so-called "bad" characteristics, based on trai

In [5]:
# Replace "|||||" with newlines or space
def preprocess_fn(example):
    cleaned_doc = example["document"].replace("|||||", "\n")
    return {
        "document": cleaned_doc,
        "summary": example["summary"]
    }

# Apply to all splits
dataset = dataset.map(preprocess_fn)


Map: 100%|██████████| 5622/5622 [00:00<00:00, 8805.54 examples/s] 


In [6]:
# Choose your model size (can switch to t5-base or bart-base later)
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
def tokenize_fn(batch):
    # Tokenize the inputs (documents)
    model_inputs = tokenizer(
        batch["document"],
        max_length=1024,
        padding="max_length",
        truncation=True
    )

    # Tokenize the targets (summaries)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["summary"],
            max_length=256,
            padding="max_length",
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [10]:
# Batched tokenization for efficiency
tokenized_dataset = dataset.map(tokenize_fn, batched=True, remove_columns=["document", "summary"])

# Confirm a tokenized sample
print(tokenized_dataset["train"][0].keys())  # should include input_ids, attention_mask, labels


Map:   0%|          | 0/44972 [00:00<?, ? examples/s]

Map:   0%|          | 0/44972 [00:00<?, ? examples/s]


TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'max_length_target'